In [17]:
from inspect import getgeneratorstate

In [18]:
def sample_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)
    yield

In [19]:
my_cro = sample_coroutine()
print(getgeneratorstate(my_cro))

GEN_CREATED


In [13]:
my_cro

<generator object sample_coroutine at 0x10dbe0dd0>

In [20]:
next(my_cro)
print(getgeneratorstate(my_cro))

-> coroutine started
GEN_SUSPENDED


In [21]:
my_cro.send(10)
print(getgeneratorstate(my_cro))

-> coroutine received: 10
GEN_SUSPENDED


In [22]:
my_cro.close()
print(getgeneratorstate(my_cro))

GEN_CLOSED


----

# 有趣的实时平均值计算

In [28]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [29]:
avg = averager()

In [30]:
avg.send(10)

TypeError: can't send non-None value to a just-started generator

In [31]:
next(avg)

In [32]:
avg.send(1)

1.0

In [33]:
avg.send(2)

1.5

In [34]:
avg.send(3)

2.0

In [35]:
avg.send(4)

2.5

In [36]:
avg.send(5)

3.0

In [37]:
avg.send(6)

3.5

In [38]:
avg.send(7)

4.0

In [39]:
avg.send(8)

4.5

In [40]:
avg.send(9)

5.0

---

# 子生成器


In [41]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager1():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total / count

In [42]:
def grouper(results, key):
    while True:
        results[key] = yield from averager1()

In [ ]:
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
            result.count, group, result.average, unit
        ))

def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)
    report(results) 